In [2]:
#Set your Imports
import os
import sys
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Add the project root to sys.path
notebook_dir = os.path.dirname(os.path.abspath('data.ipynb'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from datasharing.datasharing import DataSharingClient

In [3]:
#Connect to datasharing client $
#duckdbpath="path/to/file/nameofyourduckdbfile.duckdb"
#client = DataSharingClient(duckdb_path=duckdbpath)
client = DataSharingClient(duckdb_region='us-east-1') #Uncomment if cloest to us=east-1

It's data time!
You can query datasets in the "us-east-1" region


In [ ]:
# Define the S3 URI and local path
s3_uri = 's3://oceandatachallengebucket/public/francetax.parquet'
local_path = 'path/on/your/computer/francetax.parquet'

# Download the dataset
client.download_dataset(s3_uri, local_path)


In [5]:
#Connect to dataset
s3_uri = "s3://oceandatachallengebucket/public/francetax.parquet" #Uncomment this for us-east-1
#Change s3_uri to local_path and change the variable when calling the methid if you want to work with local files
#s3_uri = "s3://oceandatachallenges/public/francetax.parquet"
view_name = "francetax"
client.create_view(s3_uri, view_name)

View francetax created.


In [6]:
#Run this code if you want to query the view using Ibis expressions
francetaxibis = client.ibis_conn.table("francetax")
#You can name your Ibis table what you want, but it needs to match what you named your view

In [7]:
#Query a view or table and return a pandas dataframe
query = "SELECT COUNT(*) FROM francetax;"
result_df = client.query(query)
print(result_df)

   count_star()
0       1528922


In [8]:
#Query the view with an Ibis expression
#Ibis uses lazy execution, meaning your expression won't compile until you add .execute() at the end  
row_count_ibis = francetaxibis.count().execute()

print(row_count_ibis)

1528922


In [ ]:
# Query a view or table and create a duckdb table
query = "SELECT COUNT(*) FROM francetax;"
new_table_name = "francerowcount"

# Create a new table from the query
client.query(query, new_table_name)

In [ ]:
#Export a duckdb table as csv or parquet
table_names = ["francerowcount"]
output_dir = "/home/christianocean/datasharing/data"

# Export the tables to CSV format
client.export_tables(table_names, output_dir, "csv")#parquet for parquet export